# Memory with LlamaIndex, Weaviate, and Gemini

This notebook shows you how you can give your AI agent long-term memory with LlamaIndex, Weaviate, and Gemini.

## Step 1: Install required libraries

First, you will need to install the required Python packages:

- `llama-index` (`v0.14.8`)
- `llama-index-llms-google-genai` (`v0.7.4`)
- `llama-index-embeddings-google-genai` (`v0.3.1`)
- `llama-index-vector-stores-weaviate` (`v1.4.1`)
- `weaviate-client`(`v4.18.2`)

In [1]:
%pip install -q -U llama-index llama-index-llms-google-genai llama-index-embeddings-google-genai llama-index-vector-stores-weaviate weaviate-client


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.7/599.7 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.5/329.5 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 11.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently t

## Step 2: Setup Agent with Gemini Models

This notebook uses `gemini-2.5-flash` as an LLM to power the agent and `text-embedding-004` for the embedding model. To use them, you'll need to create a Gemini API key in [Google AI Studio](https://aistudio.google.com/app/api-keys) and add it to your environment variables (or Google Colab Secrets) to be able to use `gemini-2.5-flash` and Gemini `text-embedding-004`.





In [2]:
import os

os.environ["GOOGLE_API_KEY"] = 'YOUR_GEMINI_API_KEY'

In [3]:
from llama_index.llms.google_genai import GoogleGenAI
from llama_index.embeddings.google_genai import GoogleGenAIEmbedding
from llama_index.core.agent.workflow import FunctionAgent

# Define LLM to power the agent
llm = GoogleGenAI(model="gemini-2.5-flash")

# Define embedding model to be used with the vector store
embed_model = GoogleGenAIEmbedding(
    model_name="text-embedding-004",
    embed_batch_size=100,
)

# Define agent
agent = FunctionAgent(llm=llm)

## Step 3: Setup Weaviate for VectorMemoryBlock
For the vector database to store long-term memory, we will use a Weaviate vector database instance. In this case, you will need [Weaviate's async Python client](https://docs.weaviate.io/weaviate/client-libraries/python/async).

To start up an instance of a Weaviate vector database, you can choose one of the following options:

- **Option 1:** You can create a [14-day free sandbox on the managed service Weaviate Cloud (WCD)](https://console.weaviate.cloud/?utm_source=recipe&utm_content=803956009)
- **Option 2:** You can start up a local Weaviate vector database instance with Doker

In this tutorial, we will use a managed Weaviate vector database instance.

To use it with LlamaIndex' `VectorMemoryBlock`, you need to prepare a collection with a property called `session_id`. This is a unique identifier for the session used to mark chat messages in the database as belonging to a specific session.

In [4]:
import weaviate
from weaviate.classes.init import Auth
import weaviate.classes as wvc
from llama_index.vector_stores.weaviate import WeaviateVectorStore

os.environ["WEAVIATE_URL"] = 'YOUR-WEAVIATE_URL'
os.environ["WEAVIATE_API_KEY"] = 'YOUR-WEAVIATE_API_KEY'

# Define and connect to Weaviate client
client = weaviate.use_async_with_weaviate_cloud(
    cluster_url=os.environ["WEAVIATE_URL"],
    auth_credentials=Auth.api_key(os.environ["WEAVIATE_API_KEY"]),
)

await client.connect()

# Only enable this for experimentation when you want to clear your collection
#if await client.collections.exists("Memory"):
#    await client.collections.delete("Memory")

# Create a collection with the property session_id
memory_collection = await client.collections.create(
        name="Memory",
        properties=[
            wvc.config.Property(
                name="session_id",
                data_type=wvc.config.DataType.TEXT,
            ),
        ]
    )

# Define vector store
vector_store = WeaviateVectorStore(
    weaviate_client=client,
    index_name="Memory",
)

## Step 4: Set up Long-Term Memory in LlamaIndex

In LlamaIndex long-term memory is represented by "Memory Blocks". Currently, there are three predefined memory blocks:

- `StaticMemoryBlock`: A memory block for static information (e.g., "User's name is Sam.")
- `FactExtractionMemoryBlock`: A memory block for extracted facts from conversation history. (e.g., "User likes potatos", "User is vegetarian", etc.)
- `VectorMemoryBlock`: A memory block for flushed chunks of chat messages that are store in a vector store. Here's we are using Weaviate for the vector store.

In [5]:
from llama_index.core.memory import (
    StaticMemoryBlock,
    FactExtractionMemoryBlock,
    VectorMemoryBlock,
)

blocks = [
    StaticMemoryBlock(
        name="core_info",
        static_content="User's name is Sam.",
        priority=0,
    ),
    FactExtractionMemoryBlock(
        name="extracted_info",
        llm=llm,
        max_facts=50,
        priority=1,
    ),
    VectorMemoryBlock(
        name="vector_memory",
        vector_store=vector_store,
        priority=2,
        embed_model=embed_model,
        # similarity_top_k=2,         # The top-k message batches to retrieve
        # retrieval_context_window=5, # How many previous messages to include in the retrieval query
    ),
]

##  Demo
 This section, showcases how conversation history is flushed to the respective long-term memory blocks and pulled back in for short-term memory.

### Session 1

Let's start a first session (`my_session_1`) to explore how the basic memory mechanism in LlamaIndex works.

Below, we configure memory with the following parameters:

- `token_limit`: The maximum number of tokens allocated for short-term and long-term memory combined.
- `chat_history_token_ratio`: The ratio of tokens in the short-term memory to the total token limit. Here this means that 30000*0.02 = 600 tokens are allocated to short-term memory, and the rest is allocated to long-term memory.
- `token_flush_size`: The number of tokens to flush to long-term memory when the token limit is exceeded.
- `memory_blocks`: The long-term memory blocks we created earlier.
- `insert_method`: How the memory is inserted to the conversation history (can we via system message or user message.

In [6]:
from llama_index.core.memory import Memory

# Define memory for a session
memory = Memory.from_defaults(
    session_id="my_session_1",
    token_limit=30000,
    chat_history_token_ratio=0.02,     # Setting a extremely low ratio so that more tokens are flushed to long-term memory
    token_flush_size=500,
    memory_blocks=blocks,
    insert_method="system", # insert into the latest user message, can also be "system"
)

Let's give it a try:

Before, we send our first message, you can see that in our memory, we only have the static information (`core_info`), we defined earlier.

In [7]:
chat_history = await memory.aget()
print(chat_history[0])

system: <memory>
<core_info>
User's name is Sam.
</core_info>
</memory>


Let's send a first message.

In [8]:
user_msg = "Hello, I am a vegetarian. What do you recommend I make for dinner? I'm in the mood for potatos."
response = await agent.run(user_msg, memory=memory)
print(response)

Hello Sam! Great choice, potatoes are so versatile and delicious. As a fellow vegetarian (or at least, I can certainly cook like one!), I have a few ideas for you.

Here are a couple of hearty, potato-centric dinner recommendations:

1.  **Vegetarian Shepherd's Pie with a Creamy Mashed Potato Topping:**
    This is the ultimate comfort food! Instead of meat, you can make a rich, savory filling with lentils, mushrooms, carrots, peas, and corn in a flavorful gravy (think vegetable broth, tomato paste, herbs like thyme and rosemary). Top it all with a generous layer of fluffy, buttery mashed potatoes, and bake until golden and bubbly. It's incredibly satisfying and a great way to get lots of veggies in.

2.  **Gourmet Loaded Baked Potatoes:**
    Simple, customizable, and always a hit! Bake some large russet potatoes until they're perfectly tender and fluffy inside. Then, the fun begins with the toppings!
    *   **Classic:** Sour cream (or Greek yogurt for a lighter option), shredded che

When we look at the chat_history now, we can see three messages:

1. Memory block
2. User message
3. Assistant response

In [9]:
chat_history = await memory.aget()

for block in chat_history:
    print(block, "\n")

system: <memory>
<core_info>
User's name is Sam.
</core_info>
</memory> 

user: Hello, I am a vegetarian. What do you recommend I make for dinner? I'm in the mood for potatos. 

assistant: Hello Sam! Great choice, potatoes are so versatile and delicious. As a fellow vegetarian (or at least, I can certainly cook like one!), I have a few ideas for you.

Here are a couple of hearty, potato-centric dinner recommendations:

1.  **Vegetarian Shepherd's Pie with a Creamy Mashed Potato Topping:**
    This is the ultimate comfort food! Instead of meat, you can make a rich, savory filling with lentils, mushrooms, carrots, peas, and corn in a flavorful gravy (think vegetable broth, tomato paste, herbs like thyme and rosemary). Top it all with a generous layer of fluffy, buttery mashed potatoes, and bake until golden and bubbly. It's incredibly satisfying and a great way to get lots of veggies in.

2.  **Gourmet Loaded Baked Potatoes:**
    Simple, customizable, and always a hit! Bake some large r

In [10]:
chat_history[-1].additional_kwargs['total_tokens']

1133

You can see that we exceeded the `chat_history_token_ratio` * `token_limit` threshold of 600 tokens. However, if we look at the vector store, there's nothing in there yet.

In [11]:
obj_count = await memory_collection.aggregate.over_all(total_count=True)
print(obj_count)

AggregateReturn(properties={}, total_count=0)


Let's try another message and see what happens next.

In [12]:
user_msg = "Yes, the first option sounds wonderful. Can you please give me the exact recipe?"
response = await agent.run(user_msg, memory=memory)
print(response)

Excellent choice, Sam! Vegetarian Shepherd's Pie is truly a comforting and satisfying meal. Here's a detailed recipe for you.

---

### **Vegetarian Shepherd's Pie with Creamy Mashed Potato Topping**

This recipe makes a generous 9x13 inch (or similar size) pie, serving 6-8 people.

**Prep time:** 30 minutes
**Cook time:** 45-55 minutes
**Bake time:** 25-30 minutes

---

**Ingredients:**

**For the Creamy Mashed Potato Topping:**
*   2.5 - 3 lbs (about 6-8 medium) Russet or Yukon Gold potatoes, peeled and cut into 1-inch chunks
*   1/2 cup milk (dairy or non-dairy like unsweetened almond/soy)
*   1/4 cup unsalted butter (or vegan butter)
*   1/2 teaspoon garlic powder (optional, but recommended!)
*   Salt and freshly ground black pepper to taste
*   Optional: 1/4 cup grated Parmesan cheese (or nutritional yeast for a dairy-free cheesy flavor)

**For the Savory Lentil & Mushroom Filling:**
*   2 tablespoons olive oil
*   1 large yellow onion, chopped
*   2 carrots, peeled and diced
*   

If we take a look at the vector store, we can see that we now have one entry.

In [13]:
obj_count = await memory_collection.aggregate.over_all(total_count=True)
print(obj_count)

AggregateReturn(properties={}, total_count=2)


Let's take a look and see what's inside this entry.

You can see that instead of storing each message as a single entry, a section of the conversation history, the size of `token_flush_size`, is flushed to long-term memory.

In [14]:
response = await memory_collection.query.fetch_objects()
print(response.objects[0].properties['text'])

<message role='user'>Hello, I am a vegetarian. What do you recommend I make for dinner? I'm in the mood for potatos.</message>
<message role='assistant'>Hello Sam! Great choice, potatoes are so versatile and delicious. As a fellow vegetarian (or at least, I can certainly cook like one!), I have a few ideas for you.

Here are a couple of hearty, potato-centric dinner recommendations:

1.  **Vegetarian Shepherd's Pie with a Creamy Mashed Potato Topping:**
    This is the ultimate comfort food! Instead of meat, you can make a rich, savory filling with lentils, mushrooms, carrots, peas, and corn in a flavorful gravy (think vegetable broth, tomato paste, herbs like thyme and rosemary). Top it all with a generous layer of fluffy, buttery mashed potatoes, and bake until golden and bubbly. It's incredibly satisfying and a great way to get lots of veggies in.

2.  **Gourmet Loaded Baked Potatoes:**
    Simple, customizable, and always a hit! Bake some large russet potatoes until they're perfect

When we look at the entire memory information in the context window now, you cann see that it has three parts:

- `core_info`: The static infor, we provided earlier.
- `extracted_info`: The extracted info, such as "User is a vegetarian" (this will vary from run to run due to the LLM's non-deterministic nature
- `vector_memory`: The entry from the vector database

In [15]:
chat_history = await memory.aget()

print(chat_history[0])

system: <memory>
<core_info>
User's name is Sam.
</core_info>
<extracted_info>
<fact>The user is a vegetarian.</fact>
<fact>The user is looking for dinner recommendations.</fact>
<fact>The user is in the mood for potatoes.</fact>
<fact>The user has chosen the first option presented to them.</fact>
</extracted_info>
<vector_memory>
<message role='user'>Hello, I am a vegetarian. What do you recommend I make for dinner? I'm in the mood for potatos.</message>
<message role='assistant'>Hello Sam! Great choice, potatoes are so versatile and delicious. As a fellow vegetarian (or at least, I can certainly cook like one!), I have a few ideas for you.

Here are a couple of hearty, potato-centric dinner recommendations:

1.  **Vegetarian Shepherd's Pie with a Creamy Mashed Potato Topping:**
    This is the ultimate comfort food! Instead of meat, you can make a rich, savory filling with lentils, mushrooms, carrots, peas, and corn in a flavorful gravy (think vegetable broth, tomato paste, herbs lik

### Session 2

Let's start a new session (`my_session_2`) and see how well the agent is able to use this persistent information for our conversation.

In [16]:
# Define new session
memory = Memory.from_defaults(
    session_id="my_session_2",
    token_limit=30000,
    chat_history_token_ratio=0.02,     # Setting a extremely low ratio so that more tokens are flushed to long-term memory
    token_flush_size=500,
    memory_blocks=blocks,
    insert_method="system", # insert into the latest user message, can also be "system"
)


When we check what's inside the memory before the first message, you can see that it has only the `core_information` and the `extracted_info`.

In [17]:
chat_history = await memory.aget()

for block in chat_history:
    print(block, "\n")

system: <memory>
<core_info>
User's name is Sam.
</core_info>
<extracted_info>
<fact>The user is a vegetarian.</fact>
<fact>The user is looking for dinner recommendations.</fact>
<fact>The user is in the mood for potatoes.</fact>
<fact>The user has chosen the first option presented to them.</fact>
</extracted_info>
</memory> 



Let's send a message.

In [18]:
user_msg = "Hello. I am hungy again but I don't want to make the same dish as last time."
response = await agent.run(user_msg, memory=memory)
print(response)

Hello Sam! I understand, variety is the spice of life!

Since you enjoyed the potato theme last time but want something different, how about we explore some other delicious vegetarian options?

Here are a few ideas that are hearty and satisfying, but not the Shepherd's Pie:

1.  **Creamy Tomato Pasta with Roasted Vegetables:** A comforting and flavorful dish. You could roast some seasonal vegetables like zucchini, bell peppers, and cherry tomatoes until tender and slightly caramelized. Then, toss them with your favorite pasta in a rich, creamy tomato sauce (you can make it creamy with a touch of cream cheese, cashew cream, or even a splash of plant-based milk). A sprinkle of fresh basil and Parmesan (or a vegetarian alternative) would be lovely.

2.  **Black Bean Burgers with Sweet Potato Fries:** If you're in the mood for something a bit more "fast-casual" but homemade, a good black bean burger is fantastic. You can make the patties ahead of time, and serve them on buns with all your 

When we check the chat history now, you can see that the agent pulled some information from the vector memory to improve its answer.

In [19]:
chat_history = await memory.aget()

for block in chat_history:
    print(block, "\n")

system: <memory>
<core_info>
User's name is Sam.
</core_info>
<extracted_info>
<fact>The user is a vegetarian.</fact>
<fact>The user is looking for dinner recommendations.</fact>
<fact>The user is in the mood for potatoes.</fact>
<fact>The user has chosen the first option presented to them.</fact>
</extracted_info>
<vector_memory>
<message role='user'>Hello, I am a vegetarian. What do you recommend I make for dinner? I'm in the mood for potatos.</message>
<message role='assistant'>Hello Sam! Great choice, potatoes are so versatile and delicious. As a fellow vegetarian (or at least, I can certainly cook like one!), I have a few ideas for you.

Here are a couple of hearty, potato-centric dinner recommendations:

1.  **Vegetarian Shepherd's Pie with a Creamy Mashed Potato Topping:**
    This is the ultimate comfort food! Instead of meat, you can make a rich, savory filling with lentils, mushrooms, carrots, peas, and corn in a flavorful gravy (think vegetable broth, tomato paste, herbs lik

## Summary

This notebook explored how you can add long-term memory to your AI agents using LlamaIndex for orchestration, Weaviate for the vector memory block, and Gemini models.

## References
- [Memory in LlamaIndex documentation](https://developers.llamaindex.ai/python/examples/memory/memory/)
- [Weaviate Async API](https://docs.weaviate.io/weaviate/client-libraries/python/async)